# Come on down
From Josh Berry, come on down!

You’re playing a “Price Is Right” game called Cover Up, which has contestants try to guess all five digits of the price of a brand new car. You have two numbers to choose from for the first digit, three numbers to choose from for the second digit, and so on, ending with six options for the fifth and final digit. You’re not winning any $100,000 cars in this game.

First, you lock in a guess at the entire price of the car. If you get at least one digit correct on the first guess, the correct digit(s) are highlighted and you get to replace incorrect digits on a second guess. This continues on subsequent guesses until the price is guessed correctly. But if none of the new numbers you swapped in are correct, you lose. A contestant could conceivably win the car on the first guess or with five guesses, getting one additional correct digit highlighted on each guess.

First question: If you’re guessing entirely by chance, what’s the likelihood of winning the car?

Second question: Suppose you know a little bit about cars. Specifically, you are 100 percent certain about the digit in the ten-thousands place, but have to guess the remaining four digits by chance. What’s the best strategy, and what’s the likelihood of winning the car now?

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Assume the winning number is 00000. Due to symmetry the probability when the winning number is 00000 and any other arbitrary valid number should be the same.

In [2]:
WINNING_NUMBER = np.array([0, 0, 0, 0, 0])

In [3]:
def gen_permutes():
    for a in range(2):
        for b in range(3):
            for c in range(4):
                for d in range(5):
                    for e in range(6):
                        yield a, b, c, d, e

In [4]:
def can_continue(guess, prior_guess=None):
    """Determine which guesses have atleast one correct guess that were not guessed in the prior guess."""
    if prior_guess is None:
        return np.any(guess == WINNING_NUMBER, axis=1)
    return np.any((guess == WINNING_NUMBER) & (guess != prior_guess), axis=1)
    

def iterate(x, hold=False):
    """Increment guesses."""
    x = x.copy() + 1
    for i in range(5):
        if i == 0 and hold:
            continue
        x[:, i] %= i+2
    
    return x

def use_knowledge(x):
    """Use knowledge of the digit in the 10,000 place."""
    x = x.copy()
    x[:, 0] = WINNING_NUMBER[0]
    return x

def gen_new_guess(guess, prior_guess=None, hold=False):
    """Generate a new guess."""
    new_guess = np.where(guess == WINNING_NUMBER, guess, iterate(guess, hold))
    ix = ~can_continue(guess, prior_guess=prior_guess)
    new_guess[ix] = guess[ix]
    return new_guess

In [5]:
assert np.all(can_continue(np.array([[1, 1, 1, 1, 1]])) == np.array([False]))
assert np.all(can_continue(np.array([[0, 1, 1, 1, 1]]), prior_guess=np.array([[0, 1, 1, 1, 1]])) == np.array([False]))

## Estimate the probability of winning the game
The probability of winning should increase with each guess, up to a maximum of 5 guesses. We see that the probability of winning is 32.08333%

In [6]:
first_guess = np.array(list(gen_permutes()))

In [7]:
guess = first_guess.copy()
prior_guess = None
for i in range(5):
    print("Guess #{}, Probability of winning:{}".format(i+1, np.all(guess == WINNING_NUMBER, axis=1).mean()))
    old_guess = guess.copy()
    guess = gen_new_guess(guess, prior_guess=prior_guess)
    prior_guess = old_guess

Guess #1, Probability of winning:0.001388888888888889
Guess #2, Probability of winning:0.043055555555555555
Guess #3, Probability of winning:0.18194444444444444
Guess #4, Probability of winning:0.2986111111111111
Guess #5, Probability of winning:0.32083333333333336


In [8]:
# Sanity test first guess is perfect
(1/2)*(1/3)*(1/4)*(1/5)*(1/6)

0.0013888888888888887

## Estimate the probability given different strategies
Assuming you have knowledge of the 10,000th digit, you are able to use that information as a guaranteed way to guess for atleast one more round. We can evaluate the probability of winning given that you put in the right number for that digit at different rounds of guessing.

Using that information before the 2nd guess or 3rd guess is equivalent, both lead to a probability of winning of 42.777%.

In [9]:
first_guess = np.array(list(gen_permutes()))
first_guess[:, 0] = 1-WINNING_NUMBER[0]

In [10]:
for i in range(5):
    guess = first_guess.copy()
    prior_guess = None
    for j in range(5):
        if i == j:
            guess = use_knowledge(guess)
        if j == 4:
            print("Use knowledge in guess {}, Probability of winning:{}".format(i+1,
                                                                                np.all(guess == WINNING_NUMBER, axis=1).mean()))
        old_guess = guess.copy()
        guess = gen_new_guess(guess, prior_guess=prior_guess, hold=True)
        prior_guess = old_guess


Use knowledge in guess 1, Probability of winning:0.32222222222222224
Use knowledge in guess 2, Probability of winning:0.42777777777777776
Use knowledge in guess 3, Probability of winning:0.42777777777777776
Use knowledge in guess 4, Probability of winning:0.3333333333333333
Use knowledge in guess 5, Probability of winning:0.19166666666666668
